In [2]:
// Importamos la clase para DataFrame
import org.apache.spark.sql.DataFrame
// Importamos la clase para Dataset
import org.apache.spark.sql.Dataset
// Esta clase sirve para trabajar con Datasets
import spark.implicits._

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Dataset
import spark.implicits._


In [3]:
// Leemos el archivo como DataFrame, como JSON
val tweetsJSON = spark.read.json("/home/vmuser/curso/large-17-json")

tweetsJSON: org.apache.spark.sql.DataFrame = [contributors: string, coordinates: struct<coordinates: array<double>, type: string> ... 22 more fields]


In [4]:
// Mostramos el esquema del DataFrame
tweetsJSON.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- display_url: string (nullable = true)
 |    |    |    |-- expanded_url: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- id_str: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true

In [5]:
// Mostramos el primer elemento del DataFrame
tweetsJSON.first()

res1: org.apache.spark.sql.Row = [null,null,Fri Dec 21 22:53:46 +0000 2012,[WrappedArray(),null,WrappedArray([bit.ly/UWIjMN,http://bit.ly/UWIjMN,WrappedArray(48, 68),http://t.co/Nuec5Wh6]),WrappedArray()],false,null,282257583836889090,282257583836889090,null,null,null,null,null,en,null,null,false,0,false,null,<a href="http://twitterfeed.com" rel="nofollow">twitterfeed</a>,Obama meets Reid on fiscal cliff, plans remarks http://t.co/Nuec5Wh6,false,[false,Mon Mar 30 18:56:55 +0000 2009,false,false,,0,null,1995,null,67,true,27703934,27703934,false,en,94,Washington DC,Washington Examiner,null,FFFFFF,http://a0.twimg.com/profile_background_images/553522199/twitter-design-Sections1.jpg,https://si0.twimg.com/profile_background_images/553522199/twitter-design-Sections1.jpg,false,null,http://a0.tw...

In [32]:
// Mostramos los registros del atributo screen_name del atributo user, del 
// DataFrame (Ver la estructura del DataFrame)
tweetsJSON.select("user.screen_name").show()

+---------------+
|    screen_name|
+---------------+
|   washexaminer|
|narrychristmas_|
|  nicolehoran08|
|     Odemanjr15|
|       colovion|
|Coolan_n_Foolan|
|     YoSoyDalia|
|   KinderdijkNL|
|    akmalarif90|
|   organicsblue|
|  Odielicious18|
| ArticLightning|
|      skyriver1|
|      MAliciavp|
|    dave53elmer|
|     onegringaa|
|  FingerStinger|
|       mario87s|
|  brownandbella|
|   zhaiickodrjv|
+---------------+
only showing top 20 rows



In [37]:
// Creamos una clase que tendra la estructura con la que queremos trabajar el DataFrame.
// Con Option[] se define que se admiten valores nulos o vacios. 
case class Tweet(fecha_creacion: Option[String], usuario: Option[String],
                 isRT: Option[Boolean], numRT: Option[Long], texto: Option[String],
                 idioma: Option[String], tweet_sensible: Boolean, source: String,
                 favorito: Option[Boolean], numSeguidores: Option[Long])

defined class Tweet


In [38]:
// Seleccionamos las columnas indicadas del DataFrame
val tweetsDF = tweetsJSON.select("created_at", "user.screen_name",
                                "retweeted", "retweet_count", "text",
                                "lang", "possibly_sensitive", "source",
                                "retweeted_status.favorited",
                                "user.followers_count")

tweetsDF: org.apache.spark.sql.DataFrame = [created_at: string, screen_name: string ... 8 more fields]


In [36]:
// Convertimos el DataFrame(tweetsDF) a un DataSet 
// y le otorgamos la estructura definida en la clase Tweet 
val tweetsDS = tweetsDF.toDF("fecha_creacion", "usuario", "isRT",
                           "numRT", "texto", "idioma", "tweet_sensible",
                           "source", "favorito", 
                           "numSeguidores").as[Tweet]

tweetsDS: org.apache.spark.sql.Dataset[Tweet] = [fecha_creacion: string, usuario: string ... 8 more fields]


In [39]:
// Hacemos un filtrado en los registros de tweetsDS, y 
// seleccionamos las columnas indicadas.
// Obtenemos un DataFrame
val tweetSpanish = tweetsDS.filter($"idioma"==="es").select("texto", "usuario")
tweetSpanish.show()

+--------------------+---------------+
|               texto|        usuario|
+--------------------+---------------+
|RT @Clapsico: OTR...|      MAliciavp|
|RT @atienzajc: Ll...|       mario87s|
|@onedirecshon htt...| runforawarzone|
|Que Mi Media Nara...|NicolCardenas11|
|"@globovision: Ob...|       greichis|
|No pega que tenga...|juliannerxbedwe|
|Larry aun existe ...|     LWWY_LT_1D|
|RT @MiPrincesoMal...|    Ladyandboss|
|Si, http://t.co/H...|      Tururunes|
|Me piro a sobar q...|      elbabas11|
|RT @JulianGottig1...|     mati_q1000|
|RT @Anne375: #FF ...|       Karnoumz|
|@jossymi28 aww! T...| EXOPlanetSpain|
|Obama está hablan...|     FINANCEXXI|
|me ha tocado medi...|Jero5realmadrid|
|Menos buscas más ...|MauriCristiano_|
|1.7 Cubic-Ft Refr...|   workkitchens|
|Necesito verla no...|      Jesusraxd|
|https://t.co/aNOw...|     Betchakian|
|Yo cuando ponen l...|    sugardududu|
+--------------------+---------------+
only showing top 20 rows



tweetSpanish: org.apache.spark.sql.DataFrame = [texto: string, usuario: string]


In [40]:
// Seleccionamos los registros del atributo text del atributo hashtags del atributo entities,
// del DataFrame tweetsJSON (Ver la estructura del DataFrame)
// Y los convertimos a Arrays de Strings
// Obtenemos un DataSet.
val hashtag = tweetsJSON.select("entities.hashtags.text").as[Array[String]]
hashtag.show()

+--------------------+
|                text|
+--------------------+
|                  []|
|                  []|
|                  []|
|                  []|
|[nra, tcot, WAR, ...|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|                  []|
|[reasonsobamaislate]|
+--------------------+
only showing top 20 rows



hashtag: org.apache.spark.sql.Dataset[Array[String]] = [text: array<string>]


In [26]:
// Hacemos un filtrado al DataSet hashtag, en cada uno de sus elementos (Arrays de Strings)
// Solo obtenemos los arrays que no esten vacios (su tamaño sea mayor a 0)
val h = hashtag.filter(p=>p.size>0)
h.show()

+--------------------+
|                text|
+--------------------+
|[nra, tcot, WAR, ...|
|[reasonsobamaislate]|
|         [bias, Nra]|
|      [Israel, Gaza]|
|[gun, PresidentOb...|
|          [Pakistan]|
|               [GAY]|
|               [NYC]|
|             [Syria]|
|[FiscalCliff, MC,...|
|           [imagine]|
|     [MUITO, TRISTE]|
|       [Pakistan, C]|
|[obama, usa, cina...|
|            [currar]|
|[FreePalestine, I...|
|            [Taurus]|
|       [JangKeunSuk]|
|           [thefive]|
|                [FF]|
+--------------------+
only showing top 20 rows



h: org.apache.spark.sql.Dataset[Array[String]] = [text: array<string>]


In [46]:
// En cada uno de los arrays, en cada string, creamos una tupla que contiene
// el string y un 1 (entero, servira para hacer el conteo de palabras).
// Obtenemos un DataSet
val pares = h.flatMap(p=>p.map(n => (n,1)))
// Convertimos el DataSet a DataFrame, le damos nombre a las columnas
// agrupamos los registros por su atributo "key".
// Vamos sumando el atributo "value" de los registros para obtener un conteo de las palabras
val cuenta = pares.toDF("key", "value").groupBy("key").agg(sum("value"))
cuenta.show()

+----------------+----------+
|             key|sum(value)|
+----------------+----------+
|         igiveup|         1|
|           Drink|         1|
|         jewelry|        66|
|             art|        34|
|      Videogames|         2|
|          outfit|        12|
|              LT|        23|
|           Virgo|       486|
|          HRW_AR|         1|
|         smtlive|        10|
|            hope|        10|
|   CelticsEspaña|         1|
|VisitSouthAfrica|         1|
|       YaMeCanse|         1|
|           Tyler|         6|
|           anime|         4|
|      indigenous|         2|
|             NBC|        18|
|          travel|        36|
|          online|        14|
+----------------+----------+
only showing top 20 rows



pares: org.apache.spark.sql.Dataset[(String, Int)] = [_1: string, _2: int]
cuenta: org.apache.spark.sql.DataFrame = [key: string, sum(value): bigint]


In [30]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [31]:
// Ordenamos el DataFrame "cuenta" segun los valores de la columna "sum(value)"
// de manera descendente
val cuentaOrdenada = cuenta.orderBy(desc("sum(value)"))
cuentaOrdenada.show()

+---------------+----------+
|            key|sum(value)|
+---------------+----------+
|           tcot|      2912|
|       Pakistan|      2802|
|          Obama|      1974|
|           NDAA|      1843|
|          23Dec|      1693|
|            NRA|      1538|
|          black|      1241|
|       TuitUtil|      1172|
|       teaparty|      1115|
|             p2|      1078|
|    FiscalCliff|       887|
|             FF|       858|
|          Libra|       798|
|       ChangePK|       662|
|        Imagine|       636|
| DrTahirulQadri|       636|
|BelieveAcoustic|       617|
|      Capricorn|       613|
|           news|       605|
|         1DNews|       549|
+---------------+----------+
only showing top 20 rows



cuentaOrdenada: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [key: string, sum(value): bigint]
